In [3]:
%cd ..

/home/jovyan/RQC-Robotics-tactile_sensor


In [33]:
%load_ext autoreload
%autoreload 2
import sensor_lib as sl

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [23]:
n_gauses = 30  # sum of this number of gauses constitute presure profile.
n_pic = 100000  # number of generated sempels
n_fibers = 4  # number of laers in sensor (every laers have 64 parralel fibers. In different laers fibers located in different directions)
n_rot = 1  # number of random rotation (data augmentation)
lerning_rate = 1e-4
n_epochs = 20

dir_of_data = '/content/drive/MyDrive/Colab_projects/sensor/'  # part to folder with data and results

# generation pressure_map

In [ ]:
sl.generate_pressure_map(n_pic,
                         n_fibers,
                         x=97,
                         y=97,
                         part=dir_of_data + 'generated_gauses/fresh_gauss' +
                         str(n_gauses) + '_' + str(n_pic) + '.npy')

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

# counting losses in fibers (input of nerual network)

In [ ]:
input, output, input_test, output_test = sl.sim_on_gpu(
    dir_of_data + 'fresh_gauss' + str(n_gauses) + '_' + str(n_pic) + '.npy',
    n_random_rot=n_rot,
    n_angles=n_fibers,
    batch_size_preproc=128 * 8)
input_shape = input.shape
output_shape = output.shape
dataset_b, dataset = sl.prepare_dataset_for_train(input,
                                                  output,
                                                  batch_size_fit_model=1024 * 2)
dataset_test_b, dataset_test = sl.prepare_dataset_for_train(
    input_test, output_test, batch_size_fit_model=1024 * 2)

# defining nerual network for decoding and fitting it

In [ ]:
model = sl.SensorNN4S(input_shape[1:3], output_shape[1:3])
model.build(input_shape)
model.summary()
model.compile(optimizer=tf.keras.optimizers.Adam(lerning_rate),
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['accuracy'])
model_name = 'deep_model'

In [ ]:
model.fit(dataset_b, epochs=n_epochs, verbose=1)

In [ ]:
model.save(dir_of_data + 'models_weights/model4S_' + str(n_fibers) + 'fi.nn')

# evoluate model on don't seen data

In [ ]:
model = tf.keras.models.load_model(dir_of_data + 'models_weights/model4S_' +
                                   str(n_fibers) + 'fi.nn')

In [ ]:
model.evaluate(dataset_test_b)

In [ ]:
predictions = model.predict(dataset_test_b)
predictions.shape

# saving resalts

In [ ]:
with open(
        dir_of_data + 'results/' + model_name + '_' + str(n_gauses) + 'g_' +
        str(n_fibers) + 'fi', 'wb') as f:
    np.save(f, predictions)
    np.save(f, output_test)

# look on resalts

In [ ]:
with open(
        dir_of_data + 'results/' + model_name + '_' + str(n_gauses) + 'g_' +
        str(n_fibers) + 'fi', 'rb') as f:
    predictions = np.load(f)
    output_test = np.load(f)

In [ ]:
print('len= ', len(predictions))
N = 4  # number of exampel
plt.imshow(predictions[N])
plt.show()
plt.imshow(output_test[N])
plt.show()